<a href="https://colab.research.google.com/github/SeounghwanPark/first-repository/blob/master/%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%A7%81%EC%A0%91_upload%ED%95%B4%EC%84%9C_%EC%82%AC%EC%9A%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 파일 직접 업로드해서 데이터 부르기
from google.colab import files
uploaded = files.upload()

Saving cookie_cats_for_superset.csv to cookie_cats_for_superset.csv


In [2]:
import pandas as pd
from IPython.display import display

# titanic.csv 파일 읽기
df = pd.read_csv('cookie_cats_for_superset.csv')

# DataFrame을 테이블 형태로 출력
display(df.head())

,userid,version,sum_gamerounds,retention_1,retention_7,sum_gamerounds_clipped,version_bin
0,116,gate_30,3,False,False,3,0
1,337,gate_30,38,True,False,38,0
2,377,gate_40,165,True,False,165,1
3,483,gate_40,1,False,False,1,1
4,488,gate_40,179,True,True,179,1


In [4]:
import pandas as pd
import numpy as np

# 데이터 불러오기
#.read_csv("/mnt/data/cookie_cats_for_superset.csv")

# 1. 결측치 제거 (해당 데이터에는 결측치가 없음)
df = df.dropna()

# 2. 이상치 처리: sum_gamerounds는 극단치가 존재하나, clipping된 값(sum_gamerounds_clipped)를 활용
# 추가적으로 로그 변환 컬럼 추가: 0값을 고려하여 1을 더한 후 로그변환
df["log_sum_gamerounds"] = np.log(df["sum_gamerounds"] + 1)

# 3. 파생 컬럼 추가: is_gate_40
df["is_gate_40"] = (df["version"] == "gate_40").astype(int)

# 4. 파생 컬럼 추가: retention_total (True는 1, False는 0)
df["retention_total"] = df["retention_1"].astype(int) + df["retention_7"].astype(int)

# 최종 컬럼 확인
print(df.head())

# DataFrame을 테이블 형태로 출력
display(df.head())

   userid  version  sum_gamerounds  retention_1  retention_7  \
0     116  gate_30               3        False        False   
1     337  gate_30              38         True        False   
2     377  gate_40             165         True        False   
3     483  gate_40               1        False        False   
4     488  gate_40             179         True         True   

   sum_gamerounds_clipped  version_bin  log_sum_gamerounds  is_gate_40  \
0                       3            0            1.386294           0   
1                      38            0            3.663562           0   
2                     165            1            5.111988           1   
3                       1            1            0.693147           1   
4                     179            1            5.192957           1   

   retention_total  
0                0  
1                1  
2                1  
3                0  
4                2  


,userid,version,sum_gamerounds,retention_1,retention_7,sum_gamerounds_clipped,version_bin,log_sum_gamerounds,is_gate_40,retention_total
0,116,gate_30,3,False,False,3,0,1.386294,0,0
1,337,gate_30,38,True,False,38,0,3.663562,0,1
2,377,gate_40,165,True,False,165,1,5.111988,1,1
3,483,gate_40,1,False,False,1,1,0.693147,1,0
4,488,gate_40,179,True,True,179,1,5.192957,1,2


In [6]:
# t-test 통계 분석
from scipy.stats import ttest_ind

# 버전별 그룹 설정: gate_30은 control, gate_40은 treatment
control = df[df['version'] == 'gate_30']['retention_total']
treatment = df[df['version'] == 'gate_40']['retention_total']

# t-test 수행 (분산이 같지 않다고 가정)
t_stat, p_value = ttest_ind(treatment, control, equal_var=False)
print("t-statistic:", t_stat)
print("p-value:", p_value)

t-statistic: -2.9229164785113415
p-value: 0.0034685569782168997


In [ ]:
# 생성된 listings.duckdb 파일 다운로드
from google.colab import files
files.download("listings.duckdb")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Colab에서 DuckDB 파일 만들기
!pip install duckdb

import pandas as pd
import duckdb

# CSV 불러오기
df = pd.read_csv("listings.csv.gz")

# DuckDB 파일 생성
con = duckdb.connect("listings.duckdb")
con.execute("CREATE TABLE listings AS SELECT * FROM df")
con.close()


In [ ]:
# SQLite 파일로 저장하기
import sqlite3

# SQLite DB 파일 생성
conn = sqlite3.connect("listings.db")

# DataFrame을 SQLite 테이블로 저장
df.to_sql("listings", conn, if_exists="replace", index=False)

# 연결 종료
conn.close()


In [ ]:
#SQLite 파일 다운로드하기
from google.colab import files

# listings.db 파일 다운로드
files.download("listings.db")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# 정합성에 맞지 않는 행 제거
df_clean = df[
    ((df['group'] == 'control') & (df['landing_page'] == 'old_page')) |
    ((df['group'] == 'treatment') & (df['landing_page'] == 'new_page'))
]

In [ ]:
# 중복 유저 제거
df_clean = df_clean.drop_duplicates(subset='user_id')


In [ ]:
# 날짜 및 요일 컬럼 추가
df_clean['timestamp'] = pd.to_datetime(df_clean['timestamp'])
df_clean['date'] = df_clean['timestamp'].dt.date
df_clean['day_of_week'] = df_clean['timestamp'].dt.day_name()

#그룹별 전환율 계산 (막대 차트용)
conversion_rate_by_group = df_clean.groupby('group')['converted'].mean().reset_index()
conversion_rate_by_group.columns = ['group', 'conversion_rate']


In [ ]:
# 일별 그룹별 평균 전환율 계산 (선 그래프용)
daily_conversion = df_clean.groupby(['date', 'group'])['converted'].mean().reset_index()


In [ ]:
# 요일별 유저 수 계산 (캘린더 차트용)
weekday_users = df_clean.groupby(['day_of_week', 'group'])['user_id'].nunique().reset_index()
weekday_users.columns = ['day_of_week', 'group', 'user_count']


In [ ]:
# CSV 파일로 저장 (태블로에서 불러오기용)
df_clean.to_csv('clean_ab_data.csv', index=False)
daily_conversion.to_csv('daily_conversion.csv', index=False)
weekday_users.to_csv('weekday_users.csv', index=False)


In [ ]:
# CSV 다운로드
from google.colab import files

files.download('clean_ab_data.csv')
files.download('daily_conversion.csv')
files.download('weekday_users.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# 깃허브에 파일 가져오기
!git clone https://github.com/Ukbang/Streamlit.git

Cloning into 'Streamlit'...
remote: Enumerating objects: 65, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 65 (delta 19), reused 60 (delta 14), pack-reused 0 (from 0)
Receiving objects: 100% (65/65), 1.91 MiB | 15.16 MiB/s, done.
Resolving deltas: 100% (19/19), done.


In [ ]:
# 위치 확인
!ls /content/Streamlit


data  Home.py  images  pages  README.md  requirements.txt
